In [1]:
from wurlitzer import sys_pipes
import itertools
import pandas as pd
import numpy as np
from plotnine import *

import UCP.input.parser as ucp_parser
import generic.optimization.solution_extraction as ucp_out
from UCP.relaxations.lagrangian.production_state.relaxation import (
    ProductionStateRelaxation,
)
from UCP.relaxations.lagrangian.demand_satisfaction import DemandSatRelaxation
from UCP.relaxations.lagrangian.heuristic import combinatorial_heuristic
from UCP.heuristics.commit_dispatch import commit_dispatch_heuristic
from UCP.heuristics.rolling_horizon import rolling_horizon_heuristic

from generic.series_dict import series_dict_to_array, series_dict_indexes, full_series_dict

from generic.optimization.model import OptimizationSense
from generic.optimization.dual_optimization.lagrangian_decomposition import multipliers_range, fill_multipliers

from generic.optimization.dual_optimization.algorithms.cutting_plane import CuttingPlane

from generic.optimization.dual_optimization.algorithms.subgradient import (
    ComposableSubgradientMethod,
    PolyakStepSizeRule,
    CFMDeflection,
    FixedTargetTracker,
)

from generic.optimization.dual_optimization.algorithms.subgradient.volume import (
    VolumeAlgorithm,
    BundleVolumeAlgorithm,
)

from generic.optimization.dual_optimization.combined_algorithm import CombinedAlgorithm, AlgorithmConfiguration, pretty_printer, KpiCollector, extract_kpis

import UCP.output.check_solution as ck
from generic.optimization.solution_extraction import extract_solution
from UCP.output.charts import total_production, enp_vs_eie

In [2]:
theme_set(theme_bw() + theme(figure_size=(10, 10 / 1.61)))

In [3]:
data = ucp_parser.read_instance("./UCP/data/instance_5g.ucp")

In [4]:
relaxation = ProductionStateRelaxation(data)

var_lb, var_ub = tuple(
    series_dict_to_array(**m)
    for m in multipliers_range(relaxation.dualized_constraints)
)

In [5]:
configuration = AlgorithmConfiguration(
    primal_feasibility_tolerance=1e-4,
    subgradient_tolerance=1e-6,
    sgd_iterations=4,
    cp_iterations=40,
    sgd_start=False,
    heuristic_frequency=5,
    max_gap=0.025,
    max_cp_gap=0.01,
    relaxation_solver_options={},
    cp_solver_options={},#{"options":["barr", "crossover off"]},
    max_iterations=40,
    sgd_name="Polyak+CFM"
)

combined_algorithm = CombinedAlgorithm(configuration)

In [6]:
upper_bound = np.nan
current_target = np.nan

lower_cost_estimate = data.loads["value"].sum() * data.thermal_plants["l_cost"].min()
upper_cost_estimate = data.loads["value"].sum() * data.c_ENP

fixed_tracker = FixedTargetTracker(sense=-relaxation.sense, overestimation_factor=0.2)

def target_tracker(current_value: float) -> float:
    upper_bound = combined_algorithm.best_primal_bound
    if abs(upper_bound - current_value) / abs(current_value + 1e-3) > 0.2:
        fixed_target = fixed_tracker(current_value)
        result = max(fixed_target, lower_cost_estimate)
    else:
        result = upper_bound
    current_target = result
    return result



sgd_algorithm_makers = {
    "Polyak+CFM": lambda : ComposableSubgradientMethod(
            sense=-relaxation.sense,
            var_lb=var_lb,
            var_ub=var_ub,
            step_size_fun=PolyakStepSizeRule(-relaxation.sense, target_tracker),
            deflection_fun=CFMDeflection(),
        ),
    "Volume": lambda : VolumeAlgorithm(
            sense=-relaxation.sense,
            var_lb=var_lb,
            var_ub=var_ub,
            step_size_fun=PolyakStepSizeRule(-relaxation.sense, target_tracker),
        ),
    "Bundle_Volume": lambda: BundleVolumeAlgorithm(
            sense=-relaxation.sense,
            var_lb=var_lb,
            var_ub=var_ub,
            step_size_fun=PolyakStepSizeRule(-relaxation.sense, target_tracker),
        )
}

cp = CuttingPlane(sense=-relaxation.sense, var_lb=var_lb, var_ub=var_ub)

In [7]:
def run_heuristic(primal_solutions):
    mip = 0
    commitments = [ps["s"] for ps in primal_solutions]
    fixed_model, _ = combinatorial_heuristic(
        data, commitments, combination_options=dict(mip=mip, options=["ratio","0.05", "sec", "40", "doh", "barrier","crossover 0"])
    )
    solution = extract_solution(fixed_model)
    print(
        f" => Heuristic:\tTotal cost: {solution['total_production_cost']:15.5g}\t"
        + f"Demand mismatch cost:{solution['demand_mismatch_cost']:15.5g}"
    )
    return relaxation.information_from_primal_solution(solution)

In [8]:
initial_heuristic =True
if initial_heuristic:
    ucp_model, solution = rolling_horizon_heuristic(data, 72, 72, options=["sec 25 doh"])
    primal_info = relaxation.information_from_primal_solution(solution)
    initial_multipliers = relaxation.multipliers_from_primal_solution(solution)
    initial_solution = primal_info
else:
    initial_solution = None
    initial_multipliers = fill_multipliers(relaxation.dualized_constraints, 0.0)

0 23


In [9]:
combined_algorithm.initialize(relaxation, initial_multipliers, cp, sgd_algorithm_makers[configuration.sgd_name]
,run_heuristic, initial_heuristic_solution=initial_solution)

TypeError: _exploit_heuristic_solution() missing 1 required positional argument: 'heuristic_solution'

In [ ]:
kpi_collector = KpiCollector()
print(pretty_printer.header())
stop = False
while not stop:    
    stop = combined_algorithm()
    kpi_collector.collect(combined_algorithm)
    print(pretty_printer.row(combined_algorithm))

In [ ]:
tab = kpi_collector.table()
tab = extract_kpis(tab)

In [ ]:
tab

In [ ]:
temp = pd.melt(
    tab[
        [
            "iteration",
            "dual_algorithm",
            "best_dual_bound",
            "best_primal_bound",
            "best_cp_primal_bound",
#            "primal_bound",
#            "dual_bound",
        ]
    ],
    id_vars=["iteration", "dual_algorithm"],
)

p=ggplot(temp, aes("iteration", "value")) + geom_line(aes(color="variable")) + scale_color_discrete(breaks=["best_dual_bound",
            "best_primal_bound",
            "best_cp_primal_bound",
          #  "primal_bound",
          #  "dual_bound",
        ], labels=["Best Dual", "Best Primal", "Best CP Primal",]) #"Primal", "Dual"


print(p)

In [ ]:
iteration_info = pd.DataFrame.from_records(algo_kpis)
iteration_info

In [ ]:
temp = pd.melt(
    iteration_info[
        ["iteration", "dual_bound", "best_dual_bound", "cp_best_primal_bound"]
    ],
    id_vars=["iteration"],
)

best_dual = iteration_info["best_dual_bound"].max()
best_primal = iteration_info["cp_best_primal_bound"].min()

delta = max(abs(best_dual), abs(best_primal)) * 0.25
ggplot(
    temp, aes(x="iteration", y="value", color="variable", shape="variable")
) + geom_line() + geom_point() + coord_cartesian(ylim=(best_dual - delta, best_primal + delta))

In [ ]:
from sklearn.manifold import TSNE, MDS 
from sklearn.preprocessing import StandardScaler, FunctionTransformer
from sklearn.decomposition import PCA, SparsePCA, FastICA, NMF

In [ ]:
vect_matrix = np.vstack(iteration_info["multipliers_vector"][4:].to_list())
vect_matrix.shape

X=PCA(n_components=3).fit_transform(StandardScaler().fit_transform(vect_matrix))
df = pd.DataFrame(X, columns=[f"x{i}" for i in range(X.shape[1])]).reset_index().rename(columns={"index":"iteration"})
df["iteration"]+=4
df = pd.merge(df, iteration_info[["iteration", "dual_bound"]])

temp = pd.melt(df, id_vars=["iteration", "dual_bound", "x0"])

delta_y = (temp["value"].max()- temp["value"].min())/40
delta_x = (temp["x0"].max()- temp["x0"].min())/40
temp["label_pos_y"]=temp["value"]+delta_y
temp["label_pos_x"]=temp["x0"]+delta_x
ggplot(temp, aes("x0", "value", color="dual_bound")) \
+ geom_point(size=2) \
+ geom_text(aes(label="iteration",x="label_pos_x", y="label_pos_y"), position=position_jitter(width=0.4), color="black") + facet_wrap("~variable")

In [ ]:
solution = dual_bounds_list[-1].primal_solution

In [ ]:
solution["s"].groupby(level="plant").sum()

In [ ]:
solution["p"].groupby(level="period").sum()

In [ ]:
tot_production = solution["p"].groupby(level=["period"]).sum().reset_index()

tot_production = pd.merge(tot_production, data.loads[["period", "value"]], on="period")

tot_production = tot_production.rename(columns={"value": "load"})
tot_production = pd.melt(tot_production, id_vars=["period"])
ggplot(tot_production, aes("period", "value", color="variable", linetype="variable")) \
    + geom_step(size=2) \
    + scale_color_discrete(name="Series", breaks=["load", "p"], labels=["Load", "Production"]) \
    + scale_linetype_discrete(name="Series", breaks=["load", "p"], labels=["Load", "Production"]) \
    + labs(y="Value [MW]") \
    + ggtitle("Production vs Load")

In [ ]:
demand_gap = pd.DataFrame(EIE=solution["EIE"], ENP=solution["ENP"])
demand_gap = pd.melt(demand_gap, id_vars="period", var_name="Series")

ggplot(demand_gap, aes(x="period", y="value", color="Series", linetype="Series")) \
    + geom_step(size=2) \
    + labs(x="period", y="Value [MW]") \
    + ggtitle("Demand mismatch")

In [ ]:
ggplot(solution["p"], aes("period", "production")) \
    + geom_step() \
    + facet_wrap("~plant", labeller=lambda s: f"plant #{s}") \
    + labs(x="hour", y="production [MW]") \
    + ggtitle("Hourly production per plant")

In [ ]:
avg_coef = data.thermal_plants.copy()
avg_coef["avg_coef"] = avg_coef.l_cost + (avg_coef.c_cost / avg_coef.max_power)

utilization = solution["p"].groupby("plant").agg({"production": "sum"}).reset_index(0)

utilization = utilization.merge(data.thermal_plants[["plant", "max_power"]])

utilization["utilization"] = utilization.production / (utilization.max_power * len(data.time))

temp = avg_coef[["plant", "avg_coef"]]\
    .merge(utilization, sort=True)\
    [["plant", "utilization", "avg_coef"]]

ggplot(temp, aes("avg_coef", "utilization")) \
    + geom_point(size=2) \
    + geom_text(aes(y="utilization+0.04", label="plant")) \
    + scale_y_continuous(labels=percent_format()) \
    + labs(x="Avg. Hourly cost [€/MW]", y="Utilization %", label="Plant id") \
    + ggtitle("Utilization vs Hourly cost")